In [ ]:
# Install required dependencies for TECSAS
import os
os.system('pip install -q glob2==0.7 requests pytest-shutil==1.7.0 pyBigWig urllib3==1.26.14 tqdm==4.64.1 joblib==1.2.0 ipywidgets==8.0.4 biopython')

In [ ]:
# Clone the TECSAS repository
!rm -r TECSAS/
!git clone https://github.com/ed29rice/TECSAS.git

In [ ]:
# Import the TECSAS module
import TECSAS.TECSAS as TECSAS

In [ ]:
# Import necessary libraries for data processing and model evaluation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.colors as colors
from tempfile import TemporaryDirectory
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from torch.nn import functional as F
from sklearn.metrics import confusion_matrix

In [ ]:
# Set path to training data and model parameters
dpath='./'

In [ ]:
# Define model hyperparameters and initialize the TECSAS model
# Model configuration for GM12878 with 155 experiments at 50kbp resolution
n_neigbors = 14  # Number of neighboring genomic bins on each side
n_predict = 3  # Number of bins to predict
NEXP = 155  # Number of experiments (features)
nbatches = 4000  # Number of batches for training

# Transformer architecture parameters
emsize = 128  # Embedding dimension
d_hid = 64  # Hidden dimension in feedforward network
nlayers = 2  # Number of transformer encoder layers
nhead = 8  # Number of attention heads
dropout = 0.01  # Dropout rate
nfeatures = NEXP*(2*n_neigbors+1)  # Total input features
ostates = 5  # Output states (subcompartment classes: A1, A2, B1, B2, B3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TECSAS.TECSAS(n_predict, emsize, nhead, d_hid, nlayers, nfeatures, ostates, dropout).to(device)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('Number of params:',params)

In [ ]:
# Load pre-trained model weights for GM12878
# Adjust keys to match the model architecture
dict_p=torch.load(dpath+'/bv_GM12878_155.pt',map_location=torch.device('cpu'))
tmp_dict={}
for k in dict_p.keys():
    tmp_dict['.'.join(k.split('.')[1:])]=dict_p[k]

model.load_state_dict(tmp_dict)
model.eval()  # Set model to evaluation mode

In [ ]:
# Load training information and test data from checkpoint
checkpoint = torch.load(dpath+'/training_info_set_155.pt')
epoch = checkpoint['epoch']
loss = checkpoint['best_val_loss']
train_data = checkpoint['train_data']
test_data = checkpoint['test_data']
ntest_loci = checkpoint['ntest_loci']  # Genomic loci indices for test set
loci_indx = checkpoint['loci_indx']

In [ ]:
# Count trainable parameters in the model
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [ ]:
# Helper function to extract batches from test data
def get_batch_test(source: Tensor, i: int, n_predict: int, ndxs=None ):
    # Extract input features (all experiments across neighboring bins)
    data = source[i*bptt:(i+1)*bptt,2*(n_predict-1)+1:][:,:,np.newaxis]
    # Extract target labels (subcompartment annotations)
    target = source[i*bptt:(i+1)*bptt,:2*(n_predict-1)+1]
    # Extract genomic loci indices
    indexes = ndxs[i*bptt:(i+1)*bptt]
    return data.to(device), target.to(device), indexes

In [ ]:
# Evaluate model predictions on GM12878 test set at 50kbp resolution
bptt = len(train_data)//nbatches  # Batch size
nbatches_eval=len(test_data)//bptt
l=[]  # Predictions
lt=[]  # Ground truth labels
failed_inputs=[]
failed_targets=[]
failed_pred=[]
failed_loci=[]
suc_inputs=[]
suc_targets=[]
suc_pred=[]
suc_loci=[]
with torch.no_grad():
    for batch in range(nbatches_eval):
        data, targets, batch_loci = get_batch_test(test_data, batch,n_predict=n_predict, ndxs=ntest_loci)
        if batch%10==0: print(batch, nbatches_eval, len(targets))
        # Get model predictions (argmax over subcompartment classes)
        prediction=model(data,None)[0].argmax(dim=-1)[:,n_predict-1].cpu()
        # Separate failed predictions
        idx=prediction!=targets[:,n_predict-1].cpu()
        failed_inputs.append(targets[idx,n_predict-1].cpu())
        failed_targets.append(data[idx].cpu())
        failed_pred.append(prediction[idx])
        failed_loci.append(batch_loci[idx])
        # Separate successful predictions
        idx=prediction==targets[:,n_predict-1].cpu()
        suc_inputs.append(targets[idx,n_predict-1].cpu())
        suc_targets.append(data[idx].cpu())
        suc_pred.append(prediction[idx])
        suc_loci.append(batch_loci[idx])
        l.append(prediction)
        lt.append(targets[:,n_predict-1].cpu())

In [ ]:
# Concatenate all predictions and separate successful/failed cases
failed_inputs=np.concatenate(failed_inputs)
failed_pred=np.concatenate(failed_pred)
failed_targets=np.concatenate(failed_targets)
failed_loci=np.concatenate(failed_loci)
suc_inputs=np.concatenate(suc_inputs)
suc_pred=np.concatenate(suc_pred)
suc_targets=np.concatenate(suc_targets)
suc_loci=np.concatenate(suc_loci)
l=np.concatenate(l)
lt=np.concatenate(lt)

In [ ]:
# Calculate and display overall accuracy for subcompartment prediction
print('BT Accuracy:')
print('test:',np.round(np.sum(l==lt)/len(l),4))

In [ ]:
# Generate confusion matrices for subcompartment predictions
# BT: 5-class subcompartment classification (A1, A2, B1, B2, B3)
conf_matrix_P=np.round(confusion_matrix(l,lt,normalize='true'),2)
print('BT Confusion matrix:')
print(conf_matrix_P)

# AB: Binary A/B compartment classification
conf_matrix_P=np.round(confusion_matrix(l>1,lt>1,normalize='true'),2)
print('AB Confusion matrix:')
print(conf_matrix_P)